## Processing
In this Python notebook, we implement different functions to handle data quality issues from the source files. We will still be using Pandas in order to handle the processing portion of the data pipeline. We start by loading the datasets into pandas dataframes, especially the `pricing.csv` and `users.csv` as we did not have the chance to explore them previously.

Note that running this Python notebook may result into errors as the directories for this notebook and the original file for `processing.py` are different. To see the Processing script, run `processing.py` from the original directory instead.

In [164]:
import pandas as pd

### Initial observations

In [13]:
paths = [r'..\1_DataSources\data\transactions-1.csv', r'..\1_DataSources\data\pricing-1.csv', r'..\1_DataSources\data\users-1.csv']
df_transaction = pd.read_csv(paths[0])
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   trans_id    84 non-null     int64 
 1   user_id     84 non-null     int64 
 2   product     84 non-null     object
 3   amount      84 non-null     object
 4   trans_date  83 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.4+ KB


The info tells us information that we already gathered from the [Exploration portion](..\2_Exploration\EDA.ipynb) which means processing this CSV file would require (1) correcting the value for `amount` based on the value for `product`, (2) converting `trans_date` to datetime, and (3) dropping rows with null values in `trans_date`, `user_id`, and `product`.

In [17]:
df_pricing = pd.read_csv(paths[1])
print(df_pricing.info())
print(df_pricing)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   puk      6 non-null      int64 
 1   product  6 non-null      object
 2   price    6 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 276.0+ bytes
None
           puk     product  price
0  72527273070  Samsung TV    599
1  72527273071        iPad    499
2  72527273072     Macbook   1299
3  72527273073      iPhone    999
4  72527273074  Samsung TV    599
5  72527273075      Kindle     89


The only issue with the `pricing` dataframe is that in the [Exploration portion](..\2_Exploration\EDA.ipynb), the `transactions.product` column returned 5 unique values, while we see 6 unique values in the `pricing.product` column. We also see that there are two separate entries for the Samsung TV with the same price but with different values for `puk`. Assuming that `puk` serves as some sort of `id`, we will be keeping entries with the same values but different `puk` values as maybe some other dataset will be referencing the data using said values. However, as a reference for `price` resolution for lacking `amount` values, we will be referencing the one with the higher `puk` value as the higher value may refer to a more recent update on the price. This leaves us with no processing tasks involving the `pricing` dataset.

In [18]:
df_users = pd.read_csv(paths[2])
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      5 non-null      int64 
 1   name         5 non-null      object
 2   email        5 non-null      object
 3   date_joined  5 non-null      object
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes


Lastly, we see that there are only 5 unique values for `users.user_id` despite there being 9 unique values for `transactions.user_id`. This means that there will need to be a validation check between `transactions` and `users` such that the `user_id` exists on both datasets when combining tables in the future. We also note that the `date_joined` column in the `users` dataset is not set to datetime format. This will be modified as part of the formatting process.

As a general rule, all columns will have their values checked to see if they have the correct data type assigned to them. If not, then the entries will be seen as invalid, going through either a correction as `transactions.amount` does by checking with the `pricing` dataset, or will be dropped otherwise as no other columns can reference each other. The processing format is structured as follows where there exists three processors, one for each type of file, and an overall processing script that instantiates the three processors, as well as handles bridging data between two datasets when needed.
1. process_users(filepath)
    - type_check
2. process_prices(filepath)
    - type_check
3. process_transactions(filepath)
    - type_check
4. overall_processing(dirpath)
    - process_users(filepath)
    - process_prices(filepath)
    - process_transactions(filepath)
    - correct_amount

### Processor Class
The processor class stands as a template for the data sources, the template is instantiated once per type of file which in this case is `pricing`, `transactions`, and `users` where the instantiated object can serve to handle data checking.

The processor class can be seen in the `processor.py` file where the contents will only be discussed here and running the Python notebook will not execute the code properly.

In [ ]:
import pandas as pd
class Processor:
    def __init__(self, columns, types):
        self.dataframe = pd.DataFrame(columns=columns)
        self.columns = columns
        self.types = types
    


The class is instantiated by providing the needed columns and the types that each column is supposed to have. We base the columns and types off of the dataset provided.

In [ ]:
def add_csv(self, filepath):
    new_entries = pd.read_csv(filepath)
    try:
        new_entries = self.clean_data(new_entries)
        new_entries = new_entries[self.columns]
        self.dataframe = (self.dataframe.copy() if new_entries.empty else new_entries.copy() if self.dataframe.empty
                            else pd.concat([self.dataframe, new_entries], ignore_index=True)
                            )
    except KeyError as e:
        print("insufficient columns")

The `add_csv` function serves to add the data from the CSV files into the dataframe contained in each `Processor` object. Since this serves as the main way to add data into the `Processor`, we see that this function calls the succeeding functions in the data processing portion. We start by calling the `clean_data` function to perform initial cleaning of the dataset, take only the columns that we would need in the case that there are more columns in succeeding CSV files, as well as concatenate the new entries unto the existing dataframe, when adding multiple files.

In [ ]:
new_entries = self.clean_data(new_entries)

We dive deeper into cleaning the data as this is an important part of ensuring high quality data.

In [ ]:
def clean_data(self, df):
    new_df = df

    for cur in range(len(self.columns)):
        if self.types[cur] == 'int':
            new_df[self.columns[cur]] = pd.to_numeric(new_df[self.columns[cur]], errors='coerce', downcast='integer')
            if self.columns[cur] == 'amount':
                new_df[self.columns[cur]] = new_df[self.columns[cur]].fillna(-1).astype(int)
        elif self.types[cur] == 'datetime':
            new_df[self.columns[cur]] = pd.to_datetime(new_df[self.columns[cur]], errors='coerce', format='mixed')
        else:
            new_df[self.columns[cur]] = new_df[self.columns[cur]].astype(str)

    new_df = self.drop_nulls(new_df)
    return new_df

The `clean_data` function does the basic conversion of types as well as removes null values from the dataset. This lets us easily perform operations on the dataframes later on, as well as make sure that we are dealing with data of quality. We do add a special case of plugging in a -1 for the `amount` column contained in the `transactions` files as this will let us avoid dropping the null values immediately, giving us a chance to fix the missing or incorrect `amount` column.

The other functions in the `processor.py` file are either not relevant for the current scope of processing, and will instead be tackled in future sections involving error logging, or are basic utility functions that can be called by the parent.

Error logging

In [ ]:
def check_entries(self, row):
    errors = []
    for cur in range(len(self.columns)):
        if self.types[cur] == 'datetime':
            if pd.isna(row[self.columns[cur]]):
                errors.append(str(self.columns[cur]) + ' is not datetime')
        elif self.types[cur] == 'int':
            if not isinstance(row[self.columns[cur]], int):
                if self.columns[cur] == -1:
                    errors.append(str(self.columns[cur]) + ' has an incorrect data type')
                    continue
                errors.append(str(self.columns[cur]) + ' is not ' + str(self.types[cur]))
        else:
            if not isinstance(row[self.columns[cur]], str):
                errors.append(str(self.columns[cur]) + ' is not ' + str(self.types[cur]))
    return errors

def clean_data(self, df):
    new_df = df

    for cur in range(len(self.columns)):
        if self.types[cur] == 'int':
            new_df[self.columns[cur]] = pd.to_numeric(new_df[self.columns[cur]], errors='coerce', downcast='integer')
            if self.columns[cur] == 'amount':
                new_df[self.columns[cur]] = new_df[self.columns[cur]].fillna(-1).astype(int)
        elif self.types[cur] == 'datetime':
            new_df[self.columns[cur]] = pd.to_datetime(new_df[self.columns[cur]], errors='coerce', format='mixed')
        else:
            new_df[self.columns[cur]] = new_df[self.columns[cur]].astype(str)

    new_df = self.drop_nulls(new_df)
    return new_df

Public utility functions

In [ ]:
def get_dataframe(self):
    return self.dataframe

def drop_nulls(self, df):
    return df.dropna()

### Processing Script
The processing script stands to instantiate copies of each processor, runs through the specified data directory searching for csv files of the specified format, as well as handle bridging the two datasets together.

In [ ]:
def find_csv_files(directory):
    patterns = {
        'pricing-*.csv': [],
        'transactions-*.csv': [],
        'users-*.csv': []
    }

    for root, dirs, files in os.walk(directory):
        for file in files:
            for pattern in patterns.keys():
                if glob.fnmatch.fnmatch(file, pattern):
                    file_path = os.path.join(root, file)
                    patterns[pattern].append(file_path)
    
    return patterns


The `find_csv_files` function serves to walk through each the specified directory for data sources, looking for CSV files of a similar pattern as the one provided. Once all the files and directories have been searched, the patterns dictionary is returned in order to be processed using the obtained filepaths by the `process_files` function.

In [167]:

def process_files(patterns, process_users, process_transactions, process_pricing):
    for pattern, files in patterns.items():
        for file_path in files:
            if pattern == 'pricing-*.csv':
                print(f'Processing pricing file: {file_path}')
                process_pricing.add_csv(file_path)
                
            elif pattern == 'transactions-*.csv':
                print(f'Processing transactions file: {file_path}')
                process_transactions.add_csv(file_path)
                
            elif pattern == 'users-*.csv':
                print(f'Processing users file: {file_path}')
                process_users.add_csv(file_path)

The `process_files` function takes in the patterns provided by `find_csv_files`, and the processors for each type of data source as provided by the sample. The function goes through each filepath stored in the passed pattern variable and segregates the data to each processor. Once the data is segregated and the processors are populated with the data from the CSV files, the CSV file data is processed by the `processor` class in its `add_csv` function and is ready to be used. We then go proceed to the nuances of the data sample provided, starting with the issue of having missing `amount` values in the provided `transactions` data.

In [ ]:
def correct_amount(process_transactions, process_pricing):
    unique_pricing = process_pricing.dataframe.loc[process_pricing.dataframe.groupby(['product'])['puk'].idxmax()]
    merged_df = pd.merge(process_transactions.dataframe, unique_pricing, on='product')
    desired_cols = ['trans_id', 'user_id', 'product', 'price', 'trans_date']
    merged_df = merged_df[desired_cols]
    merged_df = merged_df.rename(columns={'price': 'amount'})

    process_transactions.dataframe = merged_df
    process_transactions.clean_data(process_transactions.dataframe)
    process_transactions.type_check(process_transactions.dataframe)

The missing `amount` values are handled by the `correct_amount` function where we first pass the processors for `transactions` and `pricing` and attempt to merge the data on similar `product` values. The first issue encountered is the handling of duplicates in the `pricing` dataset where products could have the same `product` and `price` values but have different `puk` values. This would result to a cartesian join if it was merged together to the `transactions` dataframe immediately, hence we would need to group the entries in the `pricing` data by their `product` values and only use the one with the highest `puk`. Then, we proceed to merging the two dataframes and store the resulting corrected `transactions` dataframe. We first filter out the merged dataframe by the columns that we need, which are the base columns for `transactions` but this time we use `price` from `pricing` instead of `amount` since this would serve to correct the values from the original dataset. Then, we modify the dataframe and proceed to cleaning and validating that the dataframe now returns corrected values.

### Summary
1. Processing was doone by creating a `Processor` class object that can be instantiated once for every datatype.
2. Cleaning the data was implemented by converting the original datatypes to their correct types.
3. Null values that were difficult to assume were dropped.
4. The missing amount values for each entry in `transactions` were corrected by merging with the `pricing` dataset.